# 4.0 Backtrader Backtest

Run and analyze the short strategy using backtrader for realistic simulation.

**Key Fixes Applied:**

1. **Cheat-on-close** (`cheat_on_close=True`): Execute orders at current bar's close to capture day 0→1 returns (not day 1→2)

2. **Per-share commission** (`commtype=bt.CommInfoBase.COMM_FIXED`): $0.003/share, not 0.3% per trade

**Expected Results (with fixes):**
- Pure signal (no DD scaling): ~+85% return, Sharpe ~0.69
- This matches the simple backtest expectation (~+102% before fees)

In [10]:
import sys
sys.path.insert(0, '.')

import logging
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path

logging.basicConfig(level=logging.INFO, format='%(levelname)s - %(message)s')

plt.style.use('seaborn-v0_8-whitegrid')
plt.rcParams['figure.figsize'] = (14, 6)

## 1. Run Backtest

In [11]:
from trading.bt_runner import run_backtest
from trading.config import DEFAULT_CONFIG

# Configuration - Use optimized model
MODEL_PATH = Path('data/model_hyperparam_optimized.pt')
DATA_DIR = Path('data')

# Date range - TEST SET (model was trained on data before 2024-10-22)
START_DATE = '2024-10-22'
END_DATE = '2025-12-18'

# Base options
INITIAL_CASH = 100_000
MAX_SYMBOLS = None
NEWS_ONLY = True

In [12]:
# Test 1: Cheat-on-close timing comparison
# This is the KEY test - does fixing the timing lag fix the strategy?

timing_configs = [
    {
        "name": "A. Next-bar execution (WRONG timing)",
        "cheat_on_close": False,
        "max_volatility": None,
        "min_dollar_volume": None,
        "use_inverse_vol": False,
    },
    {
        "name": "B. Same-day close (CORRECT timing)", 
        "cheat_on_close": True,
        "max_volatility": None,
        "min_dollar_volume": None,
        "use_inverse_vol": False,
    },
]

print(f"Test 1: Timing comparison ({len(timing_configs)} configs)")
print("="*60)

Test 1: Timing comparison (2 configs)


## 2. Run Timing Comparison

In [ ]:
%%time

timing_results = []

for i, cfg in enumerate(timing_configs):
    print(f"\n{'='*60}")
    print(f"[{i+1}/{len(timing_configs)}] Running: {cfg['name']}")
    print('='*60)
    
    try:
        res = run_backtest(
            model_path=MODEL_PATH,
            data_dir=DATA_DIR,
            start_date=START_DATE,
            end_date=END_DATE,
            initial_cash=INITIAL_CASH,
            use_dd_scaling=False,  # Pure signal for timing comparison
            use_confidence=False,
            news_only=NEWS_ONLY,
            cheat_on_close=cfg['cheat_on_close'],
            max_volatility=cfg['max_volatility'],
            min_dollar_volume=cfg['min_dollar_volume'],
            use_inverse_vol=cfg['use_inverse_vol'],
            max_symbols=MAX_SYMBOLS,
        )
        
        timing_results.append({
            'name': cfg['name'],
            'return': res.total_return,
            'sharpe': res.sharpe_ratio,
            'max_dd': res.max_drawdown,
            'n_trades': res.n_trades,
            'final_value': res.final_value,
        })
        
        print(f"\n  Result: Return={res.total_return*100:.1f}%, Sharpe={res.sharpe_ratio:.2f}, MaxDD={res.max_drawdown*100:.1f}%")
        
    except Exception as e:
        import traceback
        print(f"  ERROR: {e}")
        traceback.print_exc()
        timing_results.append({
            'name': cfg['name'],
            'return': None,
            'sharpe': None,
            'max_dd': None,
            'n_trades': 0,
            'final_value': None,
        })

INFO - Starting backtest
INFO -   Model: data/model_hyperparam_optimized.pt
INFO -   Data: data
INFO -   Period: 2024-10-22 to 2025-12-18
INFO - Loaded model from data/model_hyperparam_optimized.pt
INFO -   Price features: 9
INFO -   Fund features: 19
INFO -   Embedding dims: 768
INFO - Loading price data from data/prices.pqt...
INFO -   Loaded 5,888,410 rows



[1/2] Running: A. Next-bar execution (WRONG timing)


INFO -   Splitting into per-symbol DataFrames...
INFO -     Processed 1,000/5,644 symbols...
INFO -     Processed 2,000/5,644 symbols...
INFO -     Processed 3,000/5,644 symbols...
INFO -     Processed 4,000/5,644 symbols...
INFO -     Processed 5,000/5,644 symbols...
INFO -   Done: 5,644 symbols loaded
INFO - Adding data feeds to cerebro...
INFO -   Processed 1,000/5,644 symbols (887 added)...
INFO -   Processed 2,000/5,644 symbols (1,787 added)...
INFO -   Processed 3,000/5,644 symbols (2,685 added)...
INFO -   Processed 4,000/5,644 symbols (3,570 added)...
INFO -   Processed 5,000/5,644 symbols (4,444 added)...
INFO -   Done: 5,019 data feeds added (625 skipped)
INFO - Loading features from data/ml_dataset.pqt...
INFO -   Loaded 2,092,929 rows, 804 columns
INFO -   Filtered to 464,188 rows for 2,765 symbols
INFO -   Filtered to news-only: 127,325 rows (27.8% of 458,094)
INFO - Computing volatility features from prices...
INFO -   Computed volatility for 5,644 symbols
INFO -   Added 

## 3. Timing Results

In [ ]:
# Timing comparison results
timing_df = pd.DataFrame(timing_results)
timing_df['return_pct'] = timing_df['return'] * 100
timing_df['max_dd_pct'] = timing_df['max_dd'] * 100

print("=" * 80)
print("TIMING COMPARISON: Does cheat-on-close fix the strategy?")
print("=" * 80)
print()
print(timing_df[['name', 'return_pct', 'sharpe', 'max_dd_pct', 'n_trades']].to_string(index=False))
print()

# Analysis
if len(timing_df) == 2 and timing_df['sharpe'].notna().all():
    wrong = timing_df.iloc[0]
    correct = timing_df.iloc[1]
    print("Analysis:")
    print(f"  Wrong timing (next-bar):  Sharpe={wrong['sharpe']:.2f}, Return={wrong['return_pct']:.1f}%")
    print(f"  Correct timing (COC):     Sharpe={correct['sharpe']:.2f}, Return={correct['return_pct']:.1f}%")
    print()
    if correct['sharpe'] > wrong['sharpe']:
        print("  => Cheat-on-close IMPROVES results (as expected)")
    else:
        print("  => Cheat-on-close did NOT improve results (unexpected)")

## 4. Risk Management with Correct Timing

Now test risk management approaches WITH correct timing (cheat_on_close=True)

In [ ]:
# Risk management configs (all with cheat_on_close=True)
risk_configs = [
    {
        "name": "1. Baseline (COC, no filters)",
        "max_volatility": None,
        "min_dollar_volume": None,
        "use_inverse_vol": False,
    },
    {
        "name": "2. Vol Filter (max 100%)",
        "max_volatility": 1.0,
        "min_dollar_volume": None,
        "use_inverse_vol": False,
    },
    {
        "name": "3. Vol Filter (max 75%)",
        "max_volatility": 0.75,
        "min_dollar_volume": None,
        "use_inverse_vol": False,
    },
    {
        "name": "4. Inverse-Vol Sizing",
        "max_volatility": None,
        "min_dollar_volume": None,
        "use_inverse_vol": True,
    },
    {
        "name": "5. Vol<100% + InvVol",
        "max_volatility": 1.0,
        "min_dollar_volume": None,
        "use_inverse_vol": True,
    },
]

print(f"Test 2: Risk management with correct timing ({len(risk_configs)} configs)")

In [ ]:
%%time

risk_results = []

for i, cfg in enumerate(risk_configs):
    print(f"\n{'='*60}")
    print(f"[{i+1}/{len(risk_configs)}] Running: {cfg['name']}")
    print('='*60)
    
    try:
        res = run_backtest(
            model_path=MODEL_PATH,
            data_dir=DATA_DIR,
            start_date=START_DATE,
            end_date=END_DATE,
            initial_cash=INITIAL_CASH,
            use_dd_scaling=False,  # Pure signal - DD scaling hurts performance
            use_confidence=False,
            news_only=NEWS_ONLY,
            cheat_on_close=True,  # Always use correct timing
            max_volatility=cfg['max_volatility'],
            min_dollar_volume=cfg['min_dollar_volume'],
            use_inverse_vol=cfg['use_inverse_vol'],
            max_symbols=MAX_SYMBOLS,
        )
        
        risk_results.append({
            'name': cfg['name'],
            'return': res.total_return,
            'sharpe': res.sharpe_ratio,
            'max_dd': res.max_drawdown,
            'n_trades': res.n_trades,
            'final_value': res.final_value,
        })
        
        print(f"\n  Result: Return={res.total_return*100:.1f}%, Sharpe={res.sharpe_ratio:.2f}, MaxDD={res.max_drawdown*100:.1f}%")
        
    except Exception as e:
        import traceback
        print(f"  ERROR: {e}")
        traceback.print_exc()
        risk_results.append({
            'name': cfg['name'],
            'return': None,
            'sharpe': None,
            'max_dd': None,
            'n_trades': 0,
            'final_value': None,
        })

In [ ]:
# Risk management results
risk_df = pd.DataFrame(risk_results)
risk_df['return_pct'] = risk_df['return'] * 100
risk_df['max_dd_pct'] = risk_df['max_dd'] * 100

print("=" * 80)
print("RISK MANAGEMENT COMPARISON (with correct timing)")
print("=" * 80)
print()
print(risk_df[['name', 'return_pct', 'sharpe', 'max_dd_pct', 'n_trades']].to_string(index=False))
print()

# Find best config
if risk_df['sharpe'].notna().any():
    best_idx = risk_df['sharpe'].idxmax()
    best = risk_df.loc[best_idx]
    print(f"Best Configuration (by Sharpe):")
    print(f"  {best['name']}")
    print(f"  Sharpe: {best['sharpe']:.2f}")
    print(f"  Return: {best['return_pct']:.1f}%")
    print(f"  Max DD: {best['max_dd_pct']:.1f}%")

In [ ]:
# Visualize results
fig, axes = plt.subplots(1, 3, figsize=(15, 5))

valid_results = risk_df[risk_df['sharpe'].notna()]

# Returns
ax = axes[0]
colors = ['red' if x < 0 else 'green' for x in valid_results['return_pct']]
ax.barh(valid_results['name'], valid_results['return_pct'], color=colors)
ax.axvline(0, color='black', linestyle='-', linewidth=0.5)
ax.set_xlabel('Return (%)')
ax.set_title('Total Return')

# Sharpe
ax = axes[1]
colors = ['red' if x < 0 else 'green' for x in valid_results['sharpe']]
ax.barh(valid_results['name'], valid_results['sharpe'], color=colors)
ax.axvline(0, color='black', linestyle='-', linewidth=0.5)
ax.set_xlabel('Sharpe Ratio')
ax.set_title('Sharpe Ratio')

# Max DD
ax = axes[2]
ax.barh(valid_results['name'], valid_results['max_dd_pct'], color='darkred')
ax.set_xlabel('Max Drawdown (%)')
ax.set_title('Maximum Drawdown')

plt.tight_layout()
plt.show()

## 5. Summary

**Key Findings:**

1. **Timing is critical**: Without `cheat_on_close`, we capture day 1→2 returns instead of day 0→1, destroying the signal
2. **Commission type matters**: Must use `COMM_FIXED` for per-share fees, otherwise 0.003 = 0.3% per trade
3. **DD scaling hurts**: Locks in losses during drawdowns, preventing recovery
4. **Real performance**: ~+85% return, Sharpe ~0.69 over 14 months with $0.003/share fees